# <center>Proyecto 2</center>
<center><b>Modelación y Simulación</b></center>
<center>Javier Anleu Alegría - 17149</center>

<center>Andrea Argüello - 17801</center>

## Problema a resolver
Si un volumen fijo $V$ contiene una mezcla espacialmente uniforme de $N$ especies químicas que pueden interactuar a través de $M$ canales de reacciones química especificados, entonces, dado el número de moléculas de cada especie presentes en algún momento inicial, ¿cuáles serán estos niveles de población molecular en cualquier momento posterior? La manera tradicional de resolver este problema es por medio de $N$ ecuaciones diferenciales determinadas por las estructuras y constantes de la razón de los $M$ canales de reacciones químicas; sin embargo, este acercamiento asume que la evolución en el tiempo de un sistema químicamente reactivo es _continuo_ y _determinístico,_ cuando en la vida real este es discreto y no determinístico. Entonces es más apropiado representar el mismo por medio de una simulación _discreta_ y _estocástica_ (Gillespie, 1977).

Ya que el sistema estará en equilibrio térmico, las moléculas siempre estarán distribuidas de manera aleatoria y uniforme dentro del volumen V. Por ende, la probabilidad de que el centro de una molécula arbitraria $S_2$ esté dentro del "volumen de colisión" $\delta V_{\text{coll}}=\pi r_{12}^2v_{12}\delta t$ (donde $r_{12}=r_1+r_2$, que es la distancia entre los centros entre otra molécula $S_1$ y $S_2$ y $v_{12}$ es la velocidad relativa de $S_1$ respecto a $S_2$) en el tiempo $t$ estará dado por la razón $\frac{\delta V_{\text{coll}}}{V}$.

Lo que nos interesa entonces es: $P(X_1, X_2, ..., X_N ; t) \equiv$ la probabilidad de que en un tiempo $t$ hayan $X_1$ moléculas de la especie $S_1$, $X_2$ de la especie $S_2$, ... , y $X_N$ de la especie $S_N$ en V}

## Aplicación
El algoritmo para simular la evolución con tiempo estocástico de un sistema de eracciones químicas estará dado por los siguientes pasos:
0. Defínase los valores de las $M$ constantes de reacción, $c_1, ..., c_M$ y de las $N$ poblaciones de moléculas iniciales $X_1, ..., X_N$. $t=0$ y $n=0$. 
1. Calcular y guardar las $M$ cantidades $a_1 = h_1 c_1, ..., a_M = h_M c_M$ para los números la población molecular actual, donde $h_v \equiv$ el número $R_v$ de distintas combinaciones de reactantes moleculares en el estado $X_1, ..., X_N$. Adicionalmente, calcular y guardar $a_0 = \sum_{v=1}^M a_v$
2. Generar dos números aleatorios $r_1$ y $r_2$ utilizando un generador de números aleatorios uniformes de intervalo unitario, y calcular $\tau = \frac{1}{a_0}\ln{(1/r_1)}$ y $\mu \in \mathbb{Z}^+ \ni \sum_{v=1}^{\mu-1} a_v < r_2 a_0 \leq \sum_{v=1}^\mu a_v$.
3. Usando los valores de $\tau$ y $\mu$ del paso 2, $t=t+\tau$ y ajustar los niveles de población molecular $X_i$ para reflejar la ocurrencia de una reacción $R_\mu$. Luego, $n=n+1$ y regresar al paso 1.


## Implementación

## Recomendaciones

## Referencias
Gillespie, D. T. (1977). Exact stochastic simulation of coupled chemical reactions. _The Journal of Physical Chemistry, 81_ (25), pp. 2340–2361. doi:10.1021/j100540a008 